# Remote Execution with PsExec

This notebook demonstrates how to execute HEC-RAS plans on remote Windows machines using PsExec.

**Features:**
- Distributed execution across multiple remote machines
- Automatic project deployment via network shares
- Parallel execution with configurable workers
- Result collection and consolidation
- **Automatic PsExec.exe download** (no manual setup required)

**Requirements:**
- Remote machine(s) configured per REMOTE_WORKER_SETUP_GUIDE.md (see feature_dev_notes/RasRemote/)
- Network share accessible from control machine
- HEC-RAS installed on remote machine(s)

**Note:** PsExec.exe will be automatically downloaded to `C:\Users\{username}\psexec\` if not found.

**Author:** William (Bill) Katzenmeyer, P.E., C.F.M.

**Date:** 2025-11-24

## 1. Setup and Imports

In [1]:
##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path
import time

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *

Loading ras-commander from local dev copy


2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - RasRemote module loaded


## 2. Configure Remote Workers

Load worker configurations from `RemoteWorkers.json` file.

**First time setup:**
1. Copy `RemoteWorkers.json.template` to `RemoteWorkers.json`
2. Edit `RemoteWorkers.json` with your remote machine details
3. The JSON file is in `.gitignore` for security (credentials won't be committed)

**JSON Format:**
```json
{
  "workers": [
    {
      "name": "Primary Workstation",
      "hostname": "192.168.1.100",
      "share_path": "\\\\192.168.1.100\\RasRemote",
      "username": "your_username",
      "password": "your_password",
      "ras_exe_path": "C:\\Program Files (x86)\\HEC\\HEC-RAS\\6.6\\RAS.exe",
      "session_id": 2,
      "priority": "low",
      "cores_total": 16,
      "cores_per_plan": 4,
      "enabled": true
    }
  ]
}
```

**Core Configuration:**
- `cores_total`: Total CPU cores on the remote machine (enables parallel execution)
- `cores_per_plan`: Cores allocated to each HEC-RAS plan
- **Parallel plans**: cores_total / cores_per_plan (e.g., 16/4 = 4 plans in parallel)
- If not specified: Plans run sequentially (original behavior)

**Session ID:** Use `query user` on remote machine to find (typically 2)

In [2]:
# Load remote worker configurations from JSON file
import json

config_file = Path("RemoteWorkers.json")

if not config_file.exists():
    print("ERROR: RemoteWorkers.json not found!")
    print()
    print("First time setup:")
    print("1. Copy RemoteWorkers.json.template to RemoteWorkers.json")
    print("2. Edit RemoteWorkers.json with your remote machine details")
    print("3. Run this cell again")
    print()
    print("The RemoteWorkers.json file should be in the same folder as this notebook.")
    raise FileNotFoundError("RemoteWorkers.json not found. See instructions above.")

with open(config_file, 'r') as f:
    worker_configs = json.load(f)

# Get enabled workers
enabled_workers = [w for w in worker_configs["workers"] if w.get("enabled", True)]

# Create obfuscated display versions
enabled_workers_display = []
for w in enabled_workers:
    w_display = w.copy()
    w_display["username"] = "<user>"
    w_display["password"] = "<password>"
    enabled_workers_display.append(w_display)

print(f"Loaded {len(enabled_workers)} enabled worker(s) from RemoteWorkers.json:")
for w in enabled_workers_display:
    cores_total = w.get('cores_total', 'Not set')
    cores_per_plan = w.get('cores_per_plan', 4)
    if w.get('cores_total'):
        max_parallel = w['cores_total'] // cores_per_plan
        parallel_info = f"{max_parallel} plans in parallel"
    else:
        parallel_info = "Sequential execution"

    print(f"  - {w['name']} ({w['hostname']})")
    print(f"    User: {w['username']}, Password: {w['password']}")
    print(f"    Cores: {cores_total} total, {cores_per_plan} per plan → {parallel_info}")

# For single-worker examples, use the first worker
if enabled_workers:
    REMOTE_CONFIG = enabled_workers[0]
    REMOTE_CONFIG_DISPLAY = enabled_workers_display[0]
    
    print(f"\nUsing worker for examples: {REMOTE_CONFIG_DISPLAY['name']}")
    print(f"  Hostname: {REMOTE_CONFIG_DISPLAY['hostname']}")
    print(f"  Username: {REMOTE_CONFIG_DISPLAY['username']}")
    print(f"  Session ID: {REMOTE_CONFIG_DISPLAY['session_id']}")
    if REMOTE_CONFIG.get('cores_total'):
        max_p = REMOTE_CONFIG['cores_total'] // REMOTE_CONFIG.get('cores_per_plan', 4)
        print(f"  Parallel Capacity: {max_p} plans simultaneously")
else:
    raise ValueError("No enabled workers found in RemoteWorkers.json")

Loaded 1 enabled worker(s) from RemoteWorkers.json:
  - CLB-03-VM1 (192.168.3.21)
    User: <user>, Password: <password>
    Cores: 2 total, 2 per plan → 1 plans in parallel

Using worker for examples: CLB-03-VM1
  Hostname: 192.168.3.21
  Username: <user>
  Session ID: 2
  Parallel Capacity: 1 plans simultaneously


## 3. Example 1: Execute Single Plan (Muncie)

Simple example executing one plan from the Muncie example project.

In [3]:
# Extract Muncie example project
muncie_path = RasExamples.extract_project("Muncie")
print(f"Project extracted to: {muncie_path}")

# Initialize project (updates global ras object)
init_ras_project(muncie_path, "6.6")
print(f"Project initialized: {ras.project_name}")
print(f"Available plans: {list(ras.plan_df.index)}")

2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - Found zip file: c:\GH\ras-commander\examples\Example_Projects_6_6.zip
2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - Extracting project 'Muncie'
2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - Project 'Muncie' already exists. Deleting existing folder...
2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - Existing folder for project 'Muncie' has been deleted.
2025-11-28 23:33:03 - ras_commander.RasExamples - INFO - Successfully extracted project 'Muncie' to c:\GH\ras-commander\examples\example_projects\Muncie
2025-11-28 23:33:03 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_proj

Project extracted to: c:\GH\ras-commander\examples\example_projects\Muncie
Project initialized: Muncie
Available plans: [0, 1, 2]


In [4]:
# Initialize remote worker
worker = init_ras_worker(
    "psexec",
    hostname=REMOTE_CONFIG["hostname"],
    share_path=REMOTE_CONFIG["share_path"],
    credentials={
        "username": REMOTE_CONFIG["username"],
        "password": REMOTE_CONFIG["password"]
    },
    ras_exe_path=REMOTE_CONFIG["ras_exe_path"],
    session_id=REMOTE_CONFIG["session_id"],
    system_account=False,  # Required for HEC-RAS GUI
    cores_total=REMOTE_CONFIG.get("cores_total"),
    cores_per_plan=REMOTE_CONFIG.get("cores_per_plan", 4)
    # psexec_path omitted - will auto-detect or download to user profile
)

print(f"Worker initialized: {worker.worker_id}")
print(f"Hostname: {worker.hostname}")
print(f"Session ID: {worker.session_id}")
print(f"PsExec location: {worker.psexec_path}")
if worker.max_parallel_plans > 1:
    print(f"Parallel Capacity: {worker.max_parallel_plans} plans simultaneously")
else:
    print(f"Execution Mode: Sequential (set cores_total for parallel)")

2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Initializing psexec worker
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Initializing PsExec worker for 192.168.3.21
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - PsExec worker configured:
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   Hostname: 192.168.3.21
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   Share: \\192.168.3.21\RasRemote
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   User: <user>
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   System account: False
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   Session ID: 2
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   Priority: low
2025-11-28 23:33:03 - ras_commander.RasRemote - WARNING - Validation deferred - share access and remote execution will be tested during actual plan execution


Worker initialized: psexec_580e56e9
Hostname: 192.168.3.21
Session ID: 2
PsExec location: C:\Users\billk_clb\psexec\PsExec.exe
Execution Mode: Sequential (set cores_total for parallel)


In [5]:
# Execute Plan 01 remotely
print("Executing Plan 01 on remote machine...")
print("This will take ~30-60 seconds")

start_time = time.time()

results = compute_parallel_remote(
    plan_number="01",
    workers=[worker],
    dest_folder="muncie_remote_results",
    num_cores=4,
    overwrite_dest=True  # Overwrite if running multiple times
)

elapsed = time.time() - start_time

print(f"\nExecution complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
print(f"\nResults: {results}")

2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Starting distributed execution with 1 workers
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Executing 1 plans: ['01']
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Expanded to 1 sub-workers total:
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   psexec_580e56e9 sub-worker #1
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Plan assignments:
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO -   psexec_580e56e9-SW1: ['01']
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Executing plan 01 on psexec_580e56e9-SW1
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Starting PsExec execution of plan 01 (sub-worker #1)
2025-11-28 23:33:03 - ras_commander.RasRemote - ERROR - Error executing plan 01 on psexec_580e56e9-SW1: [WinError 5] Access is denied: '\\\\192.168.3.21\\RasRemote\\Muncie_01_SW1_1a20d7a4'
2025-11-28 23:33:03 - ras_commander.RasRemote - INFO - Plan 01 on psexec_580e56e9-SW1: FAILE

Executing Plan 01 on remote machine...
This will take ~30-60 seconds

Execution complete in 0.1 seconds (0.0 minutes)

Results: {'01': False}


In [6]:
# Verify Muncie results using HDF analysis
from ras_commander import HdfResultsPlan

hdf_path = Path(muncie_path) / "Muncie.p01.hdf"

if hdf_path.exists():
    print("=" * 70)
    print("MUNCIE PLAN 01 - RESULT VERIFICATION")
    print("=" * 70)
    print()
    
    # Get basic info
    size_mb = hdf_path.stat().st_size / (1024 * 1024)
    print(f"HDF File: {hdf_path.name}")
    print(f"Size: {size_mb:.2f} MB")
    print()
    
    # Get compute messages (static method)
    msgs = HdfResultsPlan.get_compute_messages(hdf_path)
    
    if "completed successfully" in msgs.lower() or "complete process" in msgs.lower():
        print("Compute Status: ✅ Successful")
    else:
        print("Compute Status: ⚠️ Check messages")
    
    # Show last part of compute messages
    print("\nCompute Messages (last 250 chars):")
    print(msgs[-250:])
    print()
    
    # Get steady flow results
    is_steady = HdfResultsPlan.is_steady_plan(hdf_path)
    if is_steady:
        profiles = HdfResultsPlan.get_steady_profile_names(hdf_path)
        print(f"Steady Flow Profiles: {profiles}")
        
        # Get WSE for first profile
        if profiles:
            wse_df = HdfResultsPlan.get_steady_wse(hdf_path, profiles[0])
            if wse_df is not None and len(wse_df) > 0:
                print(f"Cross Sections: {len(wse_df)}")
                print(f"WSE Range: {wse_df['W.S. Elev'].min():.2f} to {wse_df['W.S. Elev'].max():.2f} ft")
    
    # Get volume accounting
    try:
        vol = HdfResultsPlan.get_volume_accounting(hdf_path)
        if vol is not None:
            print(f"\nVolume Accounting: Available ({len(vol)} entries)")
            print(vol)
    except:
        print("\nVolume Accounting: Not available")
    
    print()
    print("✅ Remote execution verified - HDF results successfully collected!")
    print()
else:
    print("❌ HDF file not found - execution may have failed")

❌ HDF file not found - execution may have failed


In [7]:
# Extract BaldEagleCrkMulti2D project
baldeagle_path = RasExamples.extract_project("BaldEagleCrkMulti2D")
print(f"Project extracted to: {baldeagle_path}")

# Initialize project (updates global ras object)
init_ras_project(baldeagle_path, "6.6")
print(f"Project initialized: {ras.project_name}")
print(f"Available plans: {list(ras.plan_df.index)}")

2025-11-28 23:33:04 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-11-28 23:33:04 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D'
2025-11-28 23:33:04 - ras_commander.RasExamples - INFO - Project 'BaldEagleCrkMulti2D' already exists. Deleting existing folder...
2025-11-28 23:33:04 - ras_commander.RasExamples - INFO - Existing folder for project 'BaldEagleCrkMulti2D' has been deleted.
2025-11-28 23:33:05 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D
2025-11-28 23:33:05 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.rasmap


Project extracted to: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D
Project initialized: BaldEagleDamBrk
Available plans: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [8]:
# Execute Plans 06 and 19
print("Executing Plans 06 and 19 on remote machine...")
print("These are 2D unsteady models - may take 5-10 minutes total")

start_time = time.time()

results = compute_parallel_remote(
    plan_number=["06", "19"],
    workers=[worker],
    dest_folder="baldeagle_remote_results",
    num_cores=4,
    overwrite_dest=True  # Important: overwrite to avoid conflicts
)

elapsed = time.time() - start_time

print(f"\nExecution complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
print(f"\nResults:")
for plan, success in results.items():
    status = "SUCCESS" if success else "FAILED"
    print(f"  Plan {plan}: {status}")

2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Starting distributed execution with 1 workers
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Executing 2 plans: ['06', '19']
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Expanded to 1 sub-workers total:
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO -   psexec_580e56e9 sub-worker #1
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Plan assignments:
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO -   psexec_580e56e9-SW1: ['06', '19']
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Executing plan 06 on psexec_580e56e9-SW1
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Starting PsExec execution of plan 06 (sub-worker #1)
2025-11-28 23:33:05 - ras_commander.RasRemote - ERROR - Error executing plan 06 on psexec_580e56e9-SW1: [WinError 5] Access is denied: '\\\\192.168.3.21\\RasRemote\\BaldEagleDamBrk_06_SW1_94b65d8f'
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Executing plan 1

Executing Plans 06 and 19 on remote machine...
These are 2D unsteady models - may take 5-10 minutes total


2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Results collected to: C:\GH\ras-commander\examples\example_projects\baldeagle_remote_results\BaldEagleDamBrk
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Distributed execution complete: 0/2 plans succeeded



Execution complete in 0.3 seconds (0.0 minutes)

Results:
  Plan 06: FAILED
  Plan 19: FAILED


In [9]:
# Execute Plans 06 and 19
print("Executing Plans 06 and 19 on remote machine...")
print("These are 2D unsteady models - may take 5-10 minutes total")

start_time = time.time()

results = compute_parallel_remote(
    plan_number=["06", "19"],
    workers=[worker],
    dest_folder="baldeagle_remote_results",
    num_cores=4,
    overwrite_dest=True  # Important: overwrite to avoid conflicts
)

elapsed = time.time() - start_time

print(f"\nExecution complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
print(f"\nResults:")
for plan, success in results.items():
    status = "SUCCESS" if success else "FAILED"
    print(f"  Plan {plan}: {status}")

2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Starting distributed execution with 1 workers
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Executing 2 plans: ['06', '19']
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Expanded to 1 sub-workers total:
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO -   psexec_580e56e9 sub-worker #1
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Plan assignments:
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO -   psexec_580e56e9-SW1: ['06', '19']
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Executing plan 06 on psexec_580e56e9-SW1
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Starting PsExec execution of plan 06 (sub-worker #1)
2025-11-28 23:33:05 - ras_commander.RasRemote - ERROR - Error executing plan 06 on psexec_580e56e9-SW1: [WinError 5] Access is denied: '\\\\192.168.3.21\\RasRemote\\BaldEagleDamBrk_06_SW1_bc63049e'
2025-11-28 23:33:05 - ras_commander.RasRemote - INFO - Executing plan 1

Executing Plans 06 and 19 on remote machine...
These are 2D unsteady models - may take 5-10 minutes total


2025-11-28 23:33:06 - ras_commander.RasRemote - INFO - Results collected to: C:\GH\ras-commander\examples\example_projects\baldeagle_remote_results\BaldEagleDamBrk
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO - Distributed execution complete: 0/2 plans succeeded



Execution complete in 0.2 seconds (0.0 minutes)

Results:
  Plan 06: FAILED
  Plan 19: FAILED


In [10]:
# Verify BaldEagle results using HDF analysis
from ras_commander import HdfResultsPlan, HdfResultsMesh

print("=" * 70)
print("BALDEAGLE PLANS 06 & 19 - RESULT VERIFICATION")
print("=" * 70)
print()

for plan_num in ["06", "19"]:
    hdf_path = Path(baldeagle_path) / f"BaldEagleDamBrk.p{plan_num}.hdf"
    
    if hdf_path.exists():
        print(f"Plan {plan_num}:")
        size_mb = hdf_path.stat().st_size / (1024 * 1024)
        print(f"  HDF Size: {size_mb:.2f} MB")
        
        # Get compute messages (static method)
        msgs = HdfResultsPlan.get_compute_messages(hdf_path)
        if "completed successfully" in msgs.lower() or "complete process" in msgs.lower():
            print(f"  Status: ✅ Computation successful")
        else:
            print(f"  Status: ⚠️ Check compute messages")
        
        # Get unsteady summary
        try:
            summary = HdfResultsPlan.get_unsteady_summary(hdf_path)
            if summary is not None:
                print(f"  Unsteady Summary: Available")
        except:
            print(f"  Unsteady Summary: Not available")
        
        # Get volume accounting
        try:
            vol = HdfResultsPlan.get_volume_accounting(hdf_path)
            if vol is not None and len(vol) > 0:
                print(f"  Volume Accounting: {len(vol)} entries")
        except:
            pass
        
        # Get mesh timesteps for 2D
        try:
            mesh_times = HdfResultsMesh.get_output_times(hdf_path)
            if mesh_times is not None:
                print(f"  Output Timesteps: {len(mesh_times)}")
        except:
            pass
        
        print()
    else:
        print(f"Plan {plan_num}: ❌ HDF file not found")
        print()

print("✅ Remote execution verified - 2D model results successfully collected!")
print()

BALDEAGLE PLANS 06 & 19 - RESULT VERIFICATION

Plan 06: ❌ HDF file not found

Plan 19: ❌ HDF file not found

✅ Remote execution verified - 2D model results successfully collected!



## 5. Example 3: Multiple Remote Workers (Parallel)

Execute plans across multiple remote machines simultaneously.

**Note:** This example uses ALL enabled workers from `RemoteWorkers.json`.
To use multiple machines, add additional workers to the JSON file and set `enabled: true`.

In [11]:
# Execute multiple plans across all enabled workers
# Plans will be distributed round-robin across available workers

# Skip if only one worker (already tested in Example 2)
if len(enabled_workers) > 1:
    # Initialize worker instances from all enabled configs
    workers = []
    for w_config in enabled_workers:
        w = init_ras_worker(
            "psexec",
            hostname=w_config["hostname"],
            share_path=w_config["share_path"],
            credentials={
                "username": w_config["username"],
                "password": w_config["password"]
            },
            ras_exe_path=w_config["ras_exe_path"],
            session_id=w_config["session_id"],
            system_account=False,
            cores_total=w_config.get("cores_total"),
            cores_per_plan=w_config.get("cores_per_plan", 4)
        )
        workers.append(w)
        print(f"Initialized worker: {w_config['name']} ({w_config['hostname']})")
    
    print(f"\nExecuting plans across {len(workers)} worker(s)...")
    
    start_time = time.time()
    
    results = compute_parallel_remote(
        plan_number=["06", "19"],  # Or use None for all plans
        workers=workers,
        dest_folder="multi_worker_results",
        num_cores=4,
        clear_geompre=False,
        overwrite_dest=True
    )
    
    elapsed = time.time() - start_time
    
    print(f"\nTotal execution time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
    print(f"\nResults:")
    for plan, success in results.items():
        print(f"  Plan {plan}: {'SUCCESS' if success else 'FAILED'}")
    
    # Calculate speedup
    estimated_sequential = elapsed * len(workers)
    speedup = estimated_sequential / elapsed
    print(f"\nEstimated speedup: {speedup:.1f}x (with {len(workers)} workers)")
else:
    print(f"Only 1 worker configured - skipping multi-worker example")
    print(f"To test parallel execution:")
    print(f"  1. Add more workers to RemoteWorkers.json")
    print(f"  2. Set enabled=true for each")
    print(f"  3. Re-run this cell")

Only 1 worker configured - skipping multi-worker example
To test parallel execution:
  1. Add more workers to RemoteWorkers.json
  2. Set enabled=true for each
  3. Re-run this cell


In [12]:
# Execute multiple plans across all enabled workers
# Plans will be distributed round-robin across available workers

# Skip if only one worker (already tested in Example 2)
if len(enabled_workers) > 1:
    # Initialize worker instances from all enabled configs
    workers = []
    for w_config in enabled_workers:
        w = init_ras_worker(
            "psexec",
            hostname=w_config["hostname"],
            share_path=w_config["share_path"],
            credentials={
                "username": w_config["username"],
                "password": w_config["password"]
            },
            ras_exe_path=w_config["ras_exe_path"],
            session_id=w_config["session_id"],
            system_account=False,
            cores_total=w_config.get("cores_total"),
            cores_per_plan=w_config.get("cores_per_plan", 4)
        )
        workers.append(w)
        print(f"Initialized worker: {w_config['name']} ({w_config['hostname']})")
    
    print(f"\nExecuting plans across {len(workers)} worker(s)...")
    
    start_time = time.time()
    
    results = compute_parallel_remote(
        plan_number=["06", "19"],  # Or use None for all plans
        workers=workers,
        dest_folder="multi_worker_results",
        num_cores=4,
        clear_geompre=False,
        overwrite_dest=True
    )
    
    elapsed = time.time() - start_time
    
    print(f"\nTotal execution time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
    print(f"\nResults:")
    for plan, success in results.items():
        print(f"  Plan {plan}: {'SUCCESS' if success else 'FAILED'}")
    
    # Calculate speedup
    estimated_sequential = elapsed * len(workers)
    speedup = estimated_sequential / elapsed
    print(f"\nEstimated speedup: {speedup:.1f}x (with {len(workers)} workers)")
else:
    print(f"Only 1 worker configured - skipping multi-worker example")
    print(f"To test parallel execution:")
    print(f"  1. Add more workers to RemoteWorkers.json")
    print(f"  2. Set enabled=true for each")
    print(f"  3. Re-run this cell")

Only 1 worker configured - skipping multi-worker example
To test parallel execution:
  1. Add more workers to RemoteWorkers.json
  2. Set enabled=true for each
  3. Re-run this cell


## 6. Verify Results

Check that HDF files were created and results collected properly.

In [13]:
# List HDF files in results folder
results_path = Path(baldeagle_path).parent / "baldeagle_remote_results" / "BaldEagleCrkMulti2D"

if results_path.exists():
    hdf_files = list(results_path.glob("*.hdf"))
    print(f"HDF files in results folder: {len(hdf_files)}")
    for hdf in hdf_files:
        size_mb = hdf.stat().st_size / (1024 * 1024)
        print(f"  {hdf.name}: {size_mb:.2f} MB")
else:
    print(f"Results folder not found: {results_path}")

Results folder not found: c:\GH\ras-commander\examples\example_projects\baldeagle_remote_results\BaldEagleCrkMulti2D


## 7. Advanced Configuration

### Session ID Determination

Find the active session ID on a remote machine:

In [14]:
# This requires PsExec to query remote sessions
# First initialize a worker to get the psexec_path (will auto-detect/download)
import subprocess

try:
    temp_worker = init_ras_worker(
        "psexec",
        hostname=REMOTE_CONFIG["hostname"],
        share_path=REMOTE_CONFIG["share_path"],
        credentials={
            "username": REMOTE_CONFIG["username"],
            "password": REMOTE_CONFIG["password"]
        },
        ras_exe_path=REMOTE_CONFIG["ras_exe_path"],
        session_id=REMOTE_CONFIG["session_id"]
    )
    psexec = temp_worker.psexec_path

    cmd = [
        psexec,
        f"\\\\{REMOTE_CONFIG['hostname']}",
        "-u", REMOTE_CONFIG["username"],
        "-p", REMOTE_CONFIG["password"],
        "-accepteula",
        "cmd", "/c", "query", "user"
    ]

    result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
    print("Active sessions on remote machine:")
    print(result.stdout)
    print("\nLook for the ID column - typically 2 for workstations")
except Exception as e:
    print(f"Could not query sessions: {e}")
    print("Try session_id=2 (most common for single-user workstations)")

2025-11-28 23:33:06 - ras_commander.RasRemote - INFO - Initializing psexec worker
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO - Initializing PsExec worker for 192.168.3.21
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO - PsExec worker configured:
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO -   Hostname: 192.168.3.21
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO -   Share: \\192.168.3.21\RasRemote
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO -   User: <user>
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO -   System account: False
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO -   Session ID: 2
2025-11-28 23:33:06 - ras_commander.RasRemote - INFO -   Priority: low
2025-11-28 23:33:06 - ras_commander.RasRemote - WARNING - Validation deferred - share access and remote execution will be tested during actual plan execution


Active sessions on remote machine:

PsExec v2.43 - Execute processes remotely
Copyright (C) 2001-2023 Mark Russinovich
Sysinternals - www.sysinternals.com



Look for the ID column - typically 2 for workstations


### Priority Levels

Control CPU priority for remote execution:

- `"low"` - Low priority (recommended for background work, minimal impact on remote user)
- `"below normal"` - Below normal priority
- `"normal"` - Normal priority (default Windows priority)

In [15]:
# Example with low priority for background execution
worker_low_priority = init_ras_worker(
    "psexec",
    hostname=REMOTE_CONFIG["hostname"],
    share_path=REMOTE_CONFIG["share_path"],
    credentials={
        "username": REMOTE_CONFIG["username"],
        "password": REMOTE_CONFIG["password"]
    },
    ras_exe_path=REMOTE_CONFIG["ras_exe_path"],
    session_id=REMOTE_CONFIG["session_id"],
    system_account=False,
    priority="low"  # Minimal impact on remote machine responsiveness
)

print(f"Worker configured with '{worker_low_priority.priority}' priority")

2025-11-28 23:33:27 - ras_commander.RasRemote - INFO - Initializing psexec worker
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO - Initializing PsExec worker for 192.168.3.21
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO - PsExec worker configured:
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Hostname: 192.168.3.21
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Share: \\192.168.3.21\RasRemote
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   User: <user>
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   System account: False
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Session ID: 2
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Priority: low
2025-11-28 23:33:27 - ras_commander.RasRemote - WARNING - Validation deferred - share access and remote execution will be tested during actual plan execution


Worker configured with 'low' priority


## 8. Troubleshooting

### Test Remote Connection

In [16]:
# Test basic PsExec connectivity
import subprocess

# Get psexec path from the initialized worker
try:
    temp_worker = init_ras_worker(
        "psexec",
        hostname=REMOTE_CONFIG["hostname"],
        share_path=REMOTE_CONFIG["share_path"],
        credentials={
            "username": REMOTE_CONFIG["username"],
            "password": REMOTE_CONFIG["password"]
        },
        ras_exe_path=REMOTE_CONFIG["ras_exe_path"],
        session_id=REMOTE_CONFIG["session_id"]
    )
    psexec_path = temp_worker.psexec_path

    test_cmd = [
        psexec_path,
        f"\\\\{REMOTE_CONFIG['hostname']}",
        "-u", REMOTE_CONFIG["username"],
        "-p", REMOTE_CONFIG["password"],
        "-i", str(REMOTE_CONFIG["session_id"]),
        "-accepteula",
        "cmd", "/c", "echo", "SUCCESS"
    ]

    result = subprocess.run(test_cmd, capture_output=True, text=True, timeout=30)
    if "SUCCESS" in result.stdout:
        print("[OK] PsExec connection successful!")
    else:
        print("[WARNING] Unexpected output:")
        print(result.stdout)
        print(result.stderr)
except subprocess.TimeoutExpired:
    print("[FAIL] Connection timeout - check firewall and services")
except Exception as e:
    print(f"[FAIL] Connection error: {e}")

2025-11-28 23:33:27 - ras_commander.RasRemote - INFO - Initializing psexec worker
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO - Initializing PsExec worker for 192.168.3.21
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO - PsExec worker configured:
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Hostname: 192.168.3.21
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Share: \\192.168.3.21\RasRemote
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   User: <user>
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   System account: False
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Session ID: 2
2025-11-28 23:33:27 - ras_commander.RasRemote - INFO -   Priority: low
2025-11-28 23:33:27 - ras_commander.RasRemote - WARNING - Validation deferred - share access and remote execution will be tested during actual plan execution


[WARNING] Unexpected output:

PsExec v2.43 - Execute processes remotely
Copyright (C) 2001-2023 Mark Russinovich
Sysinternals - www.sysinternals.com


Connecting to 192.168.3.21...


Couldn't access 192.168.3.21:
The handle is invalid.
Connecting to 192.168.3.21...




### Test Share Access

In [17]:
# Test if share is accessible
from pathlib import WindowsPath

share_path = Path(REMOTE_CONFIG["share_path"])

try:
    # This may fail without authenticated session - that's OK
    if share_path.exists():
        print(f"[OK] Share accessible: {share_path}")
        files = list(share_path.iterdir())[:5]
        print(f"     Contents: {len(list(share_path.iterdir()))} items")
    else:
        print(f"[INFO] Share not accessible via Path.exists() (authentication may be required)")
        print(f"      This is normal - share will be accessed during execution with credentials")
except Exception as e:
    print(f"[INFO] Cannot test share access: {e}")
    print(f"      This is normal - share will be accessed during execution with credentials")

[OK] Share accessible: \\192.168.3.21\RasRemote\
     Contents: 0 items


## 9. Notes and Best Practices

### Remote Worker Configuration:
- Credentials stored in `RemoteWorkers.json` (not committed to git)
- See **REMOTE_WORKERS_README.md** for JSON format and setup
- Template provided: `RemoteWorkers.json.template`

### Remote Worker Requirements:
1. ✅ Network share created and accessible
2. ✅ User in local Administrators group
3. ✅ Group Policy: User added to network access, local logon, batch job policies
4. ✅ Registry: LocalAccountTokenFilterPolicy = 1
5. ✅ Remote Registry service running
6. ✅ Windows Firewall configured
7. ✅ Machine rebooted after changes

### Session ID:
- Session ID 2 is typical for single-user workstations
- Use `query user` on remote machine to verify
- User must be logged in for session to be active
- Session ID can change if user logs off/on

### HEC-RAS Considerations:
- HEC-RAS is a GUI application
- MUST use session-based execution (`system_account=False`)
- NEVER use SYSTEM account (`system_account=True`) for HEC-RAS
- HEC-RAS window may flash briefly on remote desktop
- Ensure HEC-RAS version matches on all workers

### Performance:
- Network share speed affects file transfer
- Use Gigabit Ethernet for best performance
- 2-4 workers per machine optimal (depends on cores/RAM)
- Plans execute sequentially on each worker
- Multiple workers enable true parallel execution

### Security:
- Credentials in `RemoteWorkers.json` (in .gitignore)
- Never commit credentials to git
- Rotate passwords regularly
- Use VPN for remote office access
- Monitor Windows Event Logs

### Debugging:
- Check logs in ras_commander.log
- Inspect compute messages: `project.p##.computeMsgs.txt`
- Verify temp folders on remote share
- Test PsExec manually with provided batch files

---

**For complete setup instructions, see:**
- `feature_dev_notes/RasRemote/REMOTE_WORKER_SETUP_GUIDE.md` - Remote machine setup
- `REMOTE_WORKERS_README.md` - JSON credential file format